In [1]:
from tensorflow.keras.layers import Input, Lambda, Dense, Flatten, Conv2D, MaxPooling2D
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.applications.vgg19 import VGG19
from tensorflow.keras.applications.resnet50 import preprocess_input
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img
import tensorflow as tf
import numpy as np
from glob import glob
import matplotlib.pyplot as plt

In [2]:
print(tf.__version__)

2.0.0


In [3]:
IMAGE_SIZE = [224, 224]

train_path = '../Dataset/Train'
test_path = '../Dataset/Test'

In [4]:
vgg19 = VGG19(input_shape= IMAGE_SIZE + [3], 
              weights= 'imagenet', 
              include_top= False)

In [5]:
vgg19.summary()

Model: "vgg19"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0     

In [6]:
for layer in vgg19.layers:
    layer.trainable = False

In [7]:
folders = glob('../Dataset/Train/*')

In [8]:
folders

['../Dataset/Train\\Parasite', '../Dataset/Train\\Uninfected']

In [9]:
x = Flatten()(vgg19.output)

In [10]:
prediction = Dense(len(folders), activation= 'softmax')(x)

model = Model(inputs= vgg19.input, outputs= prediction)

In [11]:
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0     

In [12]:
model = Sequential()

In [13]:
# CNN Model
model.add(Conv2D(filters= 16, kernel_size= 2, padding= "same", activation= 'relu', input_shape=(224,224,3)))
model.add(MaxPooling2D(pool_size= 2))

model.add(Conv2D(filters= 32, kernel_size= 2, padding= "same", activation= 'relu'))
model.add(MaxPooling2D(pool_size= 2))

model.add(Conv2D(filters= 16, kernel_size= 2, padding= "same", activation= 'relu'))
model.add(MaxPooling2D(pool_size= 2))

model.add(Flatten())

model.add(Dense(500, activation= 'relu'))
model.add(Dense(2, activation= 'softmax'))

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 224, 224, 16)      208       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 112, 112, 16)      0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 112, 112, 32)      2080      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 56, 56, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 56, 56, 16)        2064      
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 28, 28, 16)        0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 12544)             0

In [14]:
model.compile(
    loss= 'categorical_crossentropy',
    optimizer= 'adam',
    metrics= ['accuracy']
)

In [15]:
# Image data augmentation
train_datagen = ImageDataGenerator(rescale =1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)

test_datagen = ImageDataGenerator(rescale =1./255)

In [16]:
training_set = train_datagen.flow_from_directory('../Dataset/Train',
                                                target_size= (224, ),
                                                batch_size = 32,
                                                class_mode = 'categorical')

Found 416 images belonging to 2 classes.


In [17]:
training_set

In [18]:
test_set = test_datagen.flow_from_directory('../Dataset/Test',
                                            target_size = (224, 224),
                                            batch_size = 32,
                                            class_mode = 'categorical')

Found 134 images belonging to 2 classes.


In [19]:
r = model.fit_generator(
  training_set,
  validation_data=test_set,
  epochs=50,
  steps_per_epoch=len(training_set),
  validation_steps=len(test_set))